In [1]:
import sqlite3
import pandas as pd
import numpy as np
import gensim
from tqdm import tqdm_notebook
import pymorphy2
from googletrans import Translator
import sqlalchemy
from sqlalchemy.orm import sessionmaker, scoped_session, relationship, validates
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, String, Integer, Column, Date, Boolean, sql
from sqlalchemy.schema import ForeignKey, PrimaryKeyConstraint
from sqlalchemy.dialects import postgresql
from functools import partial
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import mozilla_headers

Using TensorFlow backend.


In [ ]:
cnn = sqlite3.connect("data")
cur = cnn.cursor()

In [ ]:
def translate(text, dest='ru'):
    translator = Translator(service_urls=['translate.google.com'])
    return translator.translate(text, dest=dest).text

In [ ]:
def translate_courses(courses):
    courses = list(map(lambda x: x[0] + ". " + x[1], courses))
    translated_courses = list(map(lambda x: translate(x[:2900]), tqdm_notebook(courses))) + list(map(lambda x: translate(x[:2900], dest="en"), tqdm_notebook(courses)))
    for course in translated_courses:
        cur.execute("insert into courses (title, text, type, url) values ('{0}', '{1}', '{2}', '{3}')"
                    .format(course.split()[0], course, 'translated', '-'))
        cnn.commit()

In [ ]:
def translate_tags(tags):
    translate_tags = list(map(lambda x: translate(x), tags))
    for tag in translate_tags:
        cur.execute("insert into tags (text) values ('{0}')".format(tag))
    return translate_tags

In [ ]:
def tags_of_vacancy(vacancy, tags):
    tags_of_vacancy = set()
    for j in range(len(tags)):
        tag = tags[j] if tags[j] != "c" and tags[j] != "с" else "&&**^^"
        tag = tag.lower()
        if tag in vacancy:
            tags_of_vacancy.add(tag)
    return tuple(tags_of_vacancy)

In [ ]:
def tags_of_course(course, tags):
    tags_of_course = set()
    for j in range(len(tags)):
        tag = tags[j] if tags[j] != "c" and tags[j] != "с" else "&&**^^"
        tag = tag.lower()
        if tag in course:
            tags_of_course.add(tag)
    return list(tags_of_course)

In [ ]:
courses = cur.execute("select * from courses").fetchall()

In [ ]:
tags = cur.execute("select * from tags").fetchall()

In [ ]:
tags = list(map(lambda x: x[0], tags))
courses = list(map(lambda x: x[0] + ". " + x[1], courses))

In [ ]:
tags_of_courses = [tags_of_course(course, tags) for course in courses]

In [ ]:
Xy = pd.DataFrame()
Xy["X"] = courses
Xy["y"] = list(map(lambda x: x if len(x) else None, tags_of_courses))
Xy = Xy.dropna()

In [ ]:
q = session.query(Vacancy)
vdata = session.execute(q).fetchall()

In [ ]:
tof = []
for i in tqdm_notebook(range(len(texts))):
    tof.append(tags_of_vacancy(texts[i], tags))